<a href="https://colab.research.google.com/github/BoredBolby/TheStudentLot/blob/main/TheStudentLot_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Hi Kolby#

In [3]:
!pip install requests pandas

In [8]:
import requests
import pandas as pd
# --- Configuration ---
# 1. Paste your API Key inside the quotes below
API_KEY = "LpI5P0VnbYzpwzTmyiCnJnbjunJfIaOjTqmklPspGhdez//JNSe/XqDasQArCcyN"
#2 This run will pull stats for 2024, Week 1 to test
YEAR = 2024
WEEK = 1

# --- SETUP ---
BASE_URL = "https://api.collegefootballdata.com"
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json"
}

# --- THE REQUEST ---
print(f"Attempting to fetch data for {YEAR} WEEk {WEEK}...")

# Using the "Player Game Stats" endpoint
# This gives total yards/TD per player

response = requests.get(
    f"{BASE_URL}/games/players",
    headers=headers,
    params={"year": YEAR, "week": WEEK}
)

# --- PROCESSING ---
if response.status_code == 200:
  data = response.json()
  print(f"Success! Downloaded stats for {len(data)} games.")

  # data comes in nested (Game -> Team -> Player -> Stat)
  #need to flatten into rows and columns like a spreadsheet

  records = []

  for game in data:
    game_id = game.get('id', 'No ID') #Safe get
    # Loop through teams
    for team in game.get('teams', []):
      # Use .get() so it won't crash if "school" is missing
      #team_name = team.get('school', 'Unknown Team')
      # Try 'school' first; if missing, try 'team'; if both missing, use 'Unknown'
      team_name = team.get('school', team.get('team', 'Unknown Team'))

      # Loop through passing, rushing, receiving categories

      # Passing
      for category in team.get('categories', []):
        stat_category = category.get('name', 'Unknown')

        # Loop through stat types
        for stat_type in category.get('types', []):
          stat_name = stat_type.get('name', 'stat')

          # Loop through athletes
          for athlete in stat_type.get('athletes', []):
            records.append({
                "Player": athlete.get('name', 'Unknown Player'),
                "Team": team_name,
                "GameID": game_id,
                "Category": stat_category,
                "Stat Type": stat_name,
                "Value": athlete.get('stat', 0)
            })
# Create the dataframe
df = pd.DataFrame(records)

# Show the first 5 rows to verify
print("\nHere is a preview of your data:")
display(df.head())


Attempting to fetch data for 2024 WEEk 1...
Success! Downloaded stats for 187 games.

Here is a preview of your data:


,Player,Team,GameID,Category,Stat Type,Value
0,Darryl Overstreet Jr.,Upper Iowa University,401655657,passing,C/ATT,12/31
1,Darryl Overstreet Jr.,Upper Iowa University,401655657,passing,YDS,121
2,Darryl Overstreet Jr.,Upper Iowa University,401655657,passing,AVG,3.9
3,Darryl Overstreet Jr.,Upper Iowa University,401655657,passing,TD,1
4,Darryl Overstreet Jr.,Upper Iowa University,401655657,passing,INT,3


In [12]:
# --- PREPARE DATA ---
# Distinguish "Passing Yards" from "Rushing Yards"

df['Full_Stat'] = df['Category'] + "_" + df['Stat Type']

# --- PIVOT ---
# Convert "Full_Stat" column into multiple columns
fantasy_df = df.pivot_table(
    index=['Player', 'Team', 'GameID'],
    columns = 'Full_Stat',
    values = 'Value',
    aggfunc = 'first' #if duplicates, take the first one
).reset_index()

#Fill empty spots with "0"
fantasy_df = fantasy_df.fillna(0)

# --- CLEAN THE NUMBERS ---
# Ensure text strings are recognized as numbers
cols_to_fix = [
    'passing_YDS', 'passing_TD', 'passing_INT', #'passing_2PT',
    'rushing_YDS', 'rushing_TD', #'rushing_2PT',
    'receiving_YDS', 'receiving_TD', 'receiving_REC', #'receiving_2PT',
    'fumbles_LOST', 'kickReturns_TD', 'puntReturns_TD', #'FR_TD'
]

for col in cols_to_fix:
  if col in fantasy_df.columns:
    fantasy_df[col] = pd.to_numeric(fantasy_df[col], errors='coerce').fillna(0)
  else: #if column missing then create as 0
      fantasy_df[col] = 0

# --- CALCULATE FANTASY POINTS ---
# Scoring Rules:
# Passing 0.04/yd, 4/TD, -2/INT, 2/2PT
# Rushing 0.1/yd, 6/TD, 2/2PT
# Receiving 0.1/yd, 6/TD, 1/rec, 2/2PT
# Other -2/FUM, 6/KRTD, 6/PRTD, 6/FRTD

fantasy_df['FantasyPoints'] = (
    (fantasy_df['passing_YDS'] * 0.04) +
    (fantasy_df['passing_TD'] * 4.0) +
    (fantasy_df['passing_INT'] * -2.0) +
    #(fantasy_df['passing_2PT'] * 2.0) +
    (fantasy_df['rushing_YDS'] * 0.1) +
    (fantasy_df['rushing_TD'] * 6.0) +
    #(fantasy_df['rushing_2PT'] * 2.0) +
    (fantasy_df['receiving_YDS'] * 0.1) +
    (fantasy_df['receiving_TD'] * 6.0) +
    (fantasy_df['receiving_REC'] * 1.0) +
    #(fantasy_df['receiving_2PT'] * 2.0) +
    (fantasy_df['fumbles_LOST'] * -2.0) +
    (fantasy_df['kickReturns_TD'] * 6.0) +
    (fantasy_df['puntReturns_TD'] * 6.0)
    # + (fantasy_df['FR_TD'] * 6.0)
    )

# --- SHOW LEADERBOARD ---
# Sort by total highest points
leaderboard = fantasy_df.sort_values(by='FantasyPoints',
                                     ascending=False)

print("Week 1 Fantasy Leaderboard:")
display(leaderboard[['Player', 'Team', 'FantasyPoints']].head(10))

Week 1 Fantasy Leaderboard:


Full_Stat,Player,Team,FantasyPoints
824,Ashton Jeanty,Boise State,66.10
10477,Tetairoa Mcmillan,Arizona,64.40
4243,Giles Jackson,Washington,51.10
1847,Cameron Skattebo,Arizona State,49.88
3695,Efton Chism III,Eastern Washington,47.30
2714,Damon Ward Jr.,North Texas,47.00
6126,John Mateer,Washington State,45.58
5613,Jaxson Dart,Ole Miss,45.42
4357,Harold Fannin,Bowling Green,45.10
8871,Nick Nash,San José State,44.04


In [7]:
# Looking at first game and the first team to find school label
first_game = data[0]
first_team = first_game['teams'][0]

print("Here are the keys (labels) available for the team:")
print(first_team.keys())

print("\nHere is the full team data:")
print(first_team)

Here are the keys (labels) available for the team:
dict_keys(['team', 'conference', 'homeAway', 'points', 'categories'])

Here is the full team data:
{'team': 'Upper Iowa University', 'conference': 'Great Lakes', 'homeAway': 'away', 'points': 7, 'categories': [{'name': 'passing', 'types': [{'name': 'C/ATT', 'athletes': [{'id': '5222060', 'name': 'Darryl Overstreet Jr.', 'stat': '12/31'}]}, {'name': 'YDS', 'athletes': [{'id': '5222060', 'name': 'Darryl Overstreet Jr.', 'stat': '121'}]}, {'name': 'AVG', 'athletes': [{'id': '5222060', 'name': 'Darryl Overstreet Jr.', 'stat': '3.9'}]}, {'name': 'TD', 'athletes': [{'id': '5222060', 'name': 'Darryl Overstreet Jr.', 'stat': '1'}]}, {'name': 'INT', 'athletes': [{'id': '5222060', 'name': 'Darryl Overstreet Jr.', 'stat': '3'}]}]}, {'name': 'rushing', 'types': [{'name': 'CAR', 'athletes': [{'id': '5097739', 'name': 'Jayden Mitchell', 'stat': '16'}, {'id': '5222066', 'name': 'RJ Wells', 'stat': '13'}, {'id': '5222063', 'name': 'Aboubacar Barry', '

In [10]:
# List all unique "Full_Stat" names
print(df['Full_Stat'].unique())

['passing_C/ATT' 'passing_YDS' 'passing_AVG' 'passing_TD' 'passing_INT'
 'rushing_CAR' 'rushing_YDS' 'rushing_AVG' 'rushing_TD' 'rushing_LONG'
 'receiving_REC' 'receiving_YDS' 'receiving_AVG' 'receiving_TD'
 'receiving_LONG' 'interceptions_INT' 'interceptions_YDS'
 'interceptions_TD' 'kickReturns_NO' 'kickReturns_YDS' 'kickReturns_AVG'
 'kickReturns_LONG' 'kickReturns_TD' 'puntReturns_NO' 'puntReturns_YDS'
 'puntReturns_AVG' 'puntReturns_LONG' 'puntReturns_TD' 'kicking_FG'
 'kicking_PCT' 'kicking_LONG' 'kicking_XP' 'kicking_PTS' 'punting_NO'
 'punting_YDS' 'punting_AVG' 'punting_TB' 'punting_In 20' 'punting_LONG'
 'fumbles_FUM' 'fumbles_LOST' 'fumbles_REC' 'defensive_TOT'
 'defensive_SOLO' 'defensive_SACKS' 'defensive_TFL' 'defensive_PD'
 'defensive_QB HUR' 'defensive_TD' 'passing_QBR']


In [ ]:
### To be continued